#### Customize Datasets
- 본 노트에서는 사전 정의도니 모델의 학습/테스트를 커스텀 데이터로 진행하는 법을 알려준다
- 다음 3가지 단계
  - 1. 데이터 셋 준비
  - 2. Config 준비
  - 3. 커스텀 데이터로 학습, 테스트, 추론

#### 데이터 준비
- 이상적인 상황은 커스텀 데이터를 KITTI 스타일로 바꾸는 것이지만, Calibration 파일이나 3D 어노테이션 파일은 커스텀데이터에서 구하기가 쉽지 않다.

##### 기본 데이터 포맷
  
##### 1. Point Cloud Format
- 본 라이브러리는 .bin 포맷 포인트 클라우드만 학습/추론을 제공한다.
- 따라서, 학습 전에 당신의 포인트 클라우드(일반적으로 .pcd, .las)를 .bin 포맷으로 변환해야 한다.

1. Convert .pcd to .bin: https://github.com/DanielPollithy/pypcd

In [ ]:
# You can install pypcd with the following command
pip install git+https://github.com/DanielPollithy/pypcd.git

# You can use the following script to read the .pcd file 
# and convert it to .bin format for saving:
import numpy as np
from pypcd import pypcd

pcd_data = pypcd.PointCloud.from_path('point_cloud_data.pcd')
points = np.zeros([pcd_data.width, 4], dtype=np.float32)
points[:, 0] = pcd_data.pc_data['x'].copy()
points[:, 1] = pcd_data.pc_data['y'].copy()
points[:, 2] = pcd_data.pc_data['z'].copy()
points[:, 3] = pcd_data.pc_data['intensity'].copy().astype(np.float32)
with open('point_cloud_data.bin', 'wb') as f:
    f.write(points.tobytes())

2. Convert .las to .bin: The common conversion path is .las -> .pcd -> .bin, and the conversion path .las -> .pcd can be achieved through [this tool](https://github.com/Hitachi-Automotive-And-Industry-Lab/semantic-segmentation-editor).

##### 2. Label Format
- 각 Scene의 3D 박스와 카테고리 정보는 일반적으로 .txt 파일로 저장된다.
  

In [ ]:
# format: [x, y, z, dx, dy, dz, yaw, category_name]
1.23 1.42 0.23 3.96 1.65 1.55 1.56 Car
3.51 2.15 0.42 1.05 0.87 1.86 1.23 Pedestrian
...

- 3D 박스는 통합된 3D 좌표계로 저장되어야 한다(상호 통일되어 있어야 한다)

##### 3. Calibration Format


(이하 생략)

#### An example of customized dataset
- 데이터 준비가 끝다면, 데이터를 로드하기 위한 다음 파일을 생성한다.
- mmdet3d/datasets/my_dataset.py

In [ ]:
import mmengine

from mmdet3d.registry import DATASETS
from mmdet3d.datasets import Det3DDataset

@DATASETS.register_module()
class CustomDataset(Seg3DDataset):
    METAINFO = {
        'classes': ()
        'palette': []
        'seg_valid_class_ids':
        tuple(range(XX)),
        'seg_all_class_ids':
        tuple(range(XX)),
    }

    def __init__(self,
                data_root: Optional[str] = None,
                ann_fileL: str = '',
                metainfo: Optional[dict] = None,
                data_prefix: dict = dict(
                    pts = 'points',
                    img = '',
                    pts_instance_mask='',
                    pts_semantic_mask=''),
                pipeline: List[Union[dict, Callable]] = [],
                modality: dict = dict(use_lidar=True, use_camera=False),
                

                )
                